In [68]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "KETI-AIR/ke-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [69]:
import json

file_path = "C:/Users/asia/Desktop/파이널프로젝트/03_데이터전처리/순화표현모델 데이터_리라이팅완료.jsonl"

# 줄 단위 JSON 로딩
with open(file_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f if line.strip()]

In [70]:
def preprocess(example):
    context = "\n".join(example["context"][:-1])  # 마지막 발화 제외
    input_text = f"rephrase politely:\n{context}\n{example['context'][-1]}"  # 마지막 발화
    input_text = clean_text(input_text)
    target_text = clean_text(example["output"])
    return {
        "input_text": input_text,
        "target_text": target_text
    }

processed_data = [preprocess(ex) for ex in data]

for i in range(3):
    print(f"[{i}] input_text:\n{processed_data[i]['input_text']}")
    print(f"[{i}] target_text:\n{processed_data[i]['target_text']}")
    print("="*60)

[0] input_text:
rephrase politely: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴
[0] target_text:
B: 수학쌤 목소리 너무 지루하긴 해
[1] input_text:
rephrase politely: A: 급식 오늘 뭐냐 ? B: 아마 제육볶음이었을걸 ? A: 개좋다ㅋㅋ 밥 두 공기 각 B: 니 그 돼지같은 몸으로 또 퍼먹냐ㅋㅋ 역겹다 진짜
[1] target_text:
B: 먹성 진짜 좋다 너
[2] input_text:
rephrase politely: A: 걔 오늘 교복에 단추 다 풀고 왔더라 B: 걍 꾸미고 싶었나보지 A: 근데 선생님한테 안 걸린 게 신기함 B: 지 꼴에 패션리더래ㅋㅋ 토나온다 진심
[2] target_text:
B: 걔 패션에 관심 많아보이긴 해 ㅋㅋ


In [71]:
# 1단계: 전체 데이터 전처리
def clean_text(text):
    import re
    text = re.sub(r"(ㅋ){3,}", "ㅋㅋ", text)
    text = re.sub(r"(ㅎ){3,}", "ㅎㅎ", text)
    text = re.sub(r"(;){2,}", ";", text)
    text = re.sub(r"(\.{2,})", "...", text)
    text = re.sub(r"(!){2,}", "!!", text)
    text = re.sub(r"(\?){2,}", "??", text)
    text = re.sub(r"([!?.,])", r" \1 ", text)
    text = re.sub(r"([~❤💢💥💬])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def preprocess(example):
    context = "\n".join(example["context"])
    input_text = f"rephrase politely:\n{context}\n{example['input']}"
    input_text = clean_text(input_text)
    target_text = clean_text(example["output"])
    return {
        "input_text": input_text,
        "target_text": target_text
    }

# 전처리 적용
processed_data = [preprocess(ex) for ex in data]

# 결과 확인
for i in range(3):
    print(f"[{i}] input_text:\n{processed_data[i]['input_text']}")
    print(f"[{i}] target_text:\n{processed_data[i]['target_text']}")
    print("="*60)

[0] input_text:
rephrase politely: A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴
[0] target_text:
B: 수학쌤 목소리 너무 지루하긴 해
[1] input_text:
rephrase politely: A: 급식 오늘 뭐냐 ? B: 아마 제육볶음이었을걸 ? A: 개좋다ㅋㅋ 밥 두 공기 각 B: 니 그 돼지같은 몸으로 또 퍼먹냐ㅋㅋ 역겹다 진짜 B: 니 그 돼지같은 몸으로 또 퍼먹냐ㅋㅋ 역겹다 진짜
[1] target_text:
B: 먹성 진짜 좋다 너
[2] input_text:
rephrase politely: A: 걔 오늘 교복에 단추 다 풀고 왔더라 B: 걍 꾸미고 싶었나보지 A: 근데 선생님한테 안 걸린 게 신기함 B: 지 꼴에 패션리더래ㅋㅋ 토나온다 진심 B: 지 꼴에 패션리더래ㅋㅋ 토나온다 진심
[2] target_text:
B: 걔 패션에 관심 많아보이긴 해 ㅋㅋ


In [72]:
from torch.utils.data import Dataset

class RewriteDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item["input_text"]
        target_text = item["target_text"]

        # 토큰화
        model_inputs = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                target_text,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

        # squeeze로 (1, N) → (N) 차원 축소
        model_inputs = {k: v.squeeze() for k, v in model_inputs.items()}
        model_inputs["labels"] = labels["input_ids"].squeeze()

        return model_inputs

In [73]:
from sklearn.model_selection import train_test_split

# 데이터 분할
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

# Dataset 인스턴스 생성
train_dataset = RewriteDataset(train_data, tokenizer)
test_dataset = RewriteDataset(test_data, tokenizer)

In [74]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./ke-t5-rewrite-small",   # 결과 저장 디렉토리
    learning_rate=2e-5,                   # 학습률
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,                   # 체크포인트 최대 2개 저장
    num_train_epochs=3,                   # 에폭 수
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=4,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    evaluation_strategy="steps",          # 이거 꼭 넣어야 load_best_model_at_end가 작동함
    save_strategy="steps",               # 저장과 평가 전략 일치시켜야 에러 안 남
    save_steps=500,                        # 50 step마다 저장
    eval_steps=500,                        # 50 step마다 평가
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    lr_scheduler_type="linear",
    warmup_steps=100
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [75]:
trainer.train()
trainer.save_model("./ke-t5-rewrite-small")
tokenizer.save_pretrained("./ke-t5-rewrite-small")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss



예측: 아예 마찬가지입니다 마찬가지입니다原 누군가ramprampramprampramp 차질 차질 물적 평균점수 평균점수rampramprampramprampramprampramp 평균점수rampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramp 평균점수
정답: B: 옛날 말투 같아서 오히려 정겹지 않아?
---

예측: 아예 마찬가지입니다 마찬가지입니다原 김신회rampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramp
정답: B: 열심히 준비했으니 좋은 결과 나올 거야!
---

예측: 아예 마찬가지입니다 마찬가지입니다 마찬가지입니다 봐주 누군가 마아라 물적 평균점수 평균점수창업팀rampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramp
정답: B: 좀 폐쇄적인 곳도 있으니 분위기 확인해보는 게 좋겠다
---

예측: 아예 마찬가지입니다 마찬가지입니다 마찬가지다창업팀 elsewhere 평균점수 평균점수ramprampramprampramp 평균점수ramprampramp 평균점수rampramp

ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value('string'), 'references': List(Value('string'))}
Feature option 1: {'predictions': Value('string'), 'references': Value('string')},
Input predictions: ['아예', '마찬가지입니다', '마찬가지입니다原', ..., '평균점수rampramprampramprampramprampramp', '평균점수rampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramprampramp', '평균점수'],
Input references: [['B:', '옛날', '말투', '같아서', '오히려', '정겹지', '않아?']]

In [34]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def load_model_and_tokenizer(model_path="./ke-t5-rewrite-small"):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    return tokenizer, model

def load_bad_words(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

def get_bad_words_ids(bad_words, tokenizer):
    return tokenizer(bad_words, add_special_tokens=False).input_ids

def rewrite(context, input_text, model, tokenizer, bad_words_file="badwords.txt"):
    # 프롬프트 생성
    context_text = "\n".join(context)
    prompt = f"rephrase politely:\n{context_text}\n{input_text}"

    # 비속어 로딩 및 ID 변환
    bad_words = load_bad_words(bad_words_file)
    bad_words_ids = get_bad_words_ids(bad_words, tokenizer)

    # 토크나이징
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    # 생성
    output_ids = model.generate(
        inputs["input_ids"],
        max_length=64,
        min_length=10,
        num_beams=4,
        no_repeat_ngram_size=2,
        repetition_penalty=1.2,
        early_stopping=True,
        bad_words_ids=bad_words_ids
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [35]:
context = [
    "A: 오늘 진짜 졸려 죽겠다",
    "B: 어제 늦게 잤냐?",
    "A: 응 밤새서 과제함"
]
input_text = "B: 그걸 밤샘이라고 하는 것도 웃기다ㅋㅋ"

print(rewrite(context, input_text, bad_words_file="bad_words.txt"))

原原 마찬가지입니다原 민정수석 공개된다hankookilbohankookilbo 좋겠지만hankookilbo Stamfordhankookilbo 바르셀로나hankookilbo accidenthankookilbo 특활비hankookilbo크스hankookilbo 위메프hankookilbo 특수활동비hankookilbo ridehankookilbo 조치했다hankookilbo 많게는 Stamford Stamford 바르셀로나 바르셀로나 Stamford 코픽스 Stamfordgnrad Stamford 세계랭킹 StamfordNYT Stamford Bangkok Stamford Florence Stamford 일으키 Stamford Orange Stamford 윈도우 바르셀로나 코픽스 바르셀로나 분기별 Stamford 베이징올림픽 Stamford Munich Stamford Ankara Stamford 우리금융지주
